# Introduction à JDBC
 * JDBC est une API intégrée à Java qui permet un accès simple aux Systèmes de Gestion de Dases de Données Relationnelles (SGBDR). 
 * Son objectif est d'offrir une vision uniforme des SGDBR notament en s'appuyant sur SQL et en définissant des types adaptés à Java. 
 * C'est une API qui permet de se connecter à une base de données, d'évaluer des requêtes et de parcourir l'ensemble des résultats.
 * JDBC aussi adaptable pour utiliser les spécifités d'un SGBDR donné mais au prix de la portabilité.

## Système de gestion de base de données et Dataset

 * Dans le cadre de ce cours nous utiliserons le système de gestion de base données [h2](https://www.h2database.com). 
 * Il s'agit d'un SGDB écrit en Java qui peut s'executer comme un serveur indépendant ou depuis un programme Java. Il peut aussi être utilisé purement en mémoire ou avec persistence sur disque.

Pour l'utiliser il suffit de télécharger le fichier [h2-1.4.200.jar](https://repo1.maven.org/maven2/com/h2database/h2/1.4.200/h2-1.4.200.jar) depuis un entrepôt maven.

Le serveur peut être exécuté simplement avec la commande (dans un terminal, cliquer sur + en haut à gauche) suivante pour autoriser les connexions tcp et la création automatique d'une base de données lors du premier accès. H2 supporte aussi un mode de compatibilité avec POstgresQL que nous allons utiliser

```bash
java -cp /home/jovyan/javanotebook-jdbc/h2*.jar \
    org.h2.tools.Server \
        -webAllowOthers \
        -tcpAllowOthers \
        -pgAllowOthers \
        -ifNotExists 
```

Une fois le serveur lancé il est possible d'executer un Shell SQL dans un terminal. Ici en mode de compatibilité postgresql avec *user* comme login et *secret* comme password.
Dans ce cours nous allons utiliser une base de données qui représente des données de transport (des bus) au format [GTFS](https://developers.google.com/transit/gtfs/reference/#general_transit_feed_specification_reference). Le chargement initial peut être un peu long. 

```bash
java -cp /home/jovyan/javanotebook-jdbc/h2*.jar org.h2.tools.Shell \
  -url "jdbc:h2:tcp://localhost/~/testdb;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE" \
  -user user -password secret
```

In [3]:
%%shell

# java -cp /home/jovyan/javanotebook-jdbc/h2*.jar org.h2.tools.Shell -url "jdbc:h2:~/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE" -user sa 

java -cp /home/jovyan/javanotebook-jdbc/h2*.jar org.h2.tools.RunScript \
     -url "jdbc:h2:tcp://localhost/~/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE" \
     -user sa \
     -script /home/jovyan/javanotebook-jdbc/data.sql

Définissons tout d'abord simplement une classe Java pour représenter l'une des entité de l'exemple.

In [2]:
%mavenRepo projectlombok.org http://projectlombok.org/mavenrepo
%maven org.projectlombok:lombok:1.18.14

import lombok.*;
    
@Data
@Builder
@AllArgsConstructor(staticName="of")
public class Stop {
    private long id;
    private String name;
    private float lat;
    private float lon;
    private String code;
    
    public Stop(long id, String name, float lat, float lon, String code) {
        this.id = id;
        this.name = name;
        this.lat = lat;
        this.lon = lon;
        this.code = code;
    }
    public String toString() {return "{id:'"+id+",name:'"+name+"',lat:'"+lat+"'lon:"+lon+", code='"+code+"}";}
}

In [7]:
Stop stop=new Stop(1,"X",5.9333F, 43.1167F, "X");
stop

{id:'1,name:'X',lat:'5.9333'lon:43.1167, code='X}

## The JDBC Driver for postgresql
Pour se connecter à un SGBDR spécifique il faut disposer d'une implantation particulière d'un Driver (qui devra être ajouté à l'application). 
Il existe quatre familles de drivers JDBC :
* Type 1 - s'appuie sur un autre driver (par exemple JDBC-ODBC)
* Type 2 - utilise une implatation native d'un driver coté client.
* Type 3 - utilise un middleware pour convertir les appels JDBC en appel s propriétaires du SGBDR.
* Type 4 - offre une connection directe depuis une implantation Java (thin driver).

Le type 4 est le plus courant. Il est indépendant de la plateforme et offre de meilleures performances en se connectant directement à la base de données. 

## Ouverture d'une connexion
Pour ouvrir une connection, il faut charger la classe de l'implantation du Driver et ouvrir une connexion. En théorie, on devrait utiliser ```Class.forName("nom de la classe d'implantation")``` en pratique JDBC le fait automatique en fonction de l'URL lors de la connexion.

La première chose à faire est d'ajouter l'implantation du Driver pour la  base de données visée. Par exemple, en ajoutant à Maven une dépendance comme ci-dessous pour PostgreSQL.

In [8]:
%%loadFromPOM
<dependency>
    <groupId>org.postgresql</groupId>
    <artifactId>postgresql</artifactId>
    <version>42.2.11</version>
</dependency>

<dependency>
    <groupId>com.h2database</groupId>
    <artifactId>h2</artifactId>
    <version>1.4.200</version>    
</dependency>

Pour ouvrir une connexion, on utilise une URL dont la forme est spécifique à chaque SGBD par exemple pour [MySQL](https://dev.mysql.com/doc/connector-j/5.1/en/connector-j-usagenotes-connect-drivermanager.html), [PostgreSQL](https://jdbc.postgresql.org/) ou [H2](http://www.h2database.com/html/features.html) (une base de données relationnel en pur Java):
* ```jdbc:mysql://localhost:3306/myDatabase?user=username&password=password```
* ```jdbc:postgresql://localhost/myDatabase```
* ```jdbc:h2:mem:myDatabase```

Les informations d'authentification peuvent parfois être donnés dans l'URL ou lors de l'appel de la méthode ```Connection DriverManager.getConnection(jdbcURL)```.

In [9]:
import java.sql.Connection;
import java.sql.DriverManager;
//String jdbcURL="jdbc:postgresql://db:5432/dvdrental?user=postgres&password=changeme";
String jdbcURL="jdbc:h2:tcp://localhost/~/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE";

Connection connection = DriverManager.getConnection(jdbcURL,"sa","")


## Exécution de requêtes SQL
Pour exécuter des requetes SQL on utilise des instances de la classes ```Statement```.

Il existe trois types de statements :
* statement: requêtes simples
* prepared statement: requêtes précompilées
* callable statement: procédures stockées

Un statement est créé à partir d'une connection.

## Les requêtes simples
La méthode à appeler est différente suivant la nature de la requêtes SQL que l’on veut exécuter :
* Consultation (select)
  * executeQuery() On parcours les t-uples avec un ResultSet

* Mise à jour (update, insert, delete) ou gestion de la base de
données (create table,...)
  * executeUpdate() renvoie le nombre de lignes modifiées

* Type inconnu (ex. donné par un fonction sous forme de String)
ou si la requêtes peut renvoyer plusieurs résultats (procédures
stockées)
  * execute()

In [10]:
import java.sql.Statement;
import java.sql.ResultSet;
// Une requête simple
Statement statement = connection.createStatement();
String query1 = "SELECT * FROM gtfs_stops LIMIT 5";
ResultSet resultSet = statement.executeQuery(query1);

## Parcours des résultats
* executeQuery() retourne de ResultSet
* L’interface ResultSet définit les méthodes pour accéder au valeur des attributs
  * getXXX(int numéroDeColonne)
  * getXXX(String nomDeColonne)
  * XXX désigne le type Java de la valeur que l'on va récupérer (Byte, Boolean, AsciiStream, Short, String UnicodeStream, Int Bytes, BinaryStream, Long, Date, Object, Float, Time, BigDecimal,TimeStamp)
* A Noter : données volumineuses (ex. Blob)
  * Ouverture d'un flux

In [11]:
import java.util.List;
import java.util.ArrayList;
List<Stop> stops = new ArrayList<>();
while (resultSet.next()) {
        stops.add(new Stop(resultSet.getLong("stop_id"),
                resultSet.getString("stop_name"),
                resultSet.getFloat("stop_lat"),
                resultSet.getFloat("stop_lon"),
                resultSet.getString("stop_code")));
}
System.out.println(stops);

[{id:'100027,name:'Beau Vézé',lat:'43.10094'lon:6.059851, code='CABEAN}, {id:'100028,name:'Beau Vézé',lat:'43.100803'lon:6.059965, code='CABEAS}, {id:'100029,name:'Californie',lat:'43.085365'lon:6.098909, code='CACALN}, {id:'100030,name:'Californie',lat:'43.08519'lon:6.1001363, code='CACALS}, {id:'100031,name:'Canebas',lat:'43.096413'lon:6.0689964, code='CACANE}]


In [12]:
Statement statement = connection.createStatement();
String query1 = """
    INSERT INTO gtfs_agency(agency_id, agency_name, agency_url, agency_timezone, agency_lang)
    VALUES(2, 'Neverland', 'http://nowhere.fr', 'Europe/Paris','fr')""";
int numberOfChanges = statement.executeUpdate(query1);

## Les exceptions
* Erreur dans le code SQL : SQLException
* Avertissement lors de l'exécution (SQLWarning)
  * Problèmes de conversion de données (DataTruncation - sous-classe de SQLWarning)

In [13]:
//Traitement d'une erreur SQL
import java.sql.SQLException;
String wrongQuery = " SELECT * FROM Employee" ;
try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","");
            Statement statement = connection.createStatement();
            ResultSet resultSet = statement.executeQuery(wrongQuery)) {
        // Do stuff here
        } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();            
        }

org.h2.jdbc.JdbcSQLSyntaxErrorException: Table "employee" not found; SQL statement:
 SELECT * FROM Employee [42102-200]
	at org.h2.message.DbException.getJdbcSQLException(DbException.java:453)
	at org.h2.message.DbException.getJdbcSQLException(DbException.java:429)
	at org.h2.message.DbException.get(DbException.java:205)
	at org.h2.message.DbException.get(DbException.java:181)
	at org.h2.command.Parser.readTableOrView(Parser.java:7628)
	at org.h2.command.Parser.readTableFilter(Parser.java:1970)
	at org.h2.command.Parser.parseSelectFromPart(Parser.java:2827)
	at org.h2.command.Parser.parseSelect(Parser.java:2959)
	at org.h2.command.Parser.parseQuerySub(Parser.java:2817)
	at org.h2.command.Parser.parseSelectUnion(Parser.java:2649)
	at org.h2.command.Parser.parseQuery(Parser.java:2620)
	at org.h2.command.Parser.parsePrepared(Parser.java:868)
	at org.h2.command.Parser.parse(Parser.java:843)
	at org.h2.command.Parser.parse(Parser.java:815)
	at org.h2.command.Parser.prepareCommand(Parser.jav

## Un exemple complet

Voilà un exemple complet avec la classique des exceptions. Attention, quand une connection n'est plus utilisée il faut la fermer pour libérer les ressources sur le SGBD.

In [14]:
 String jdbcURL="jdbc:h2:tcp://localhost/~/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE";

        Connection connection = null;
        ResultSet resultSet = null;
        Statement statement = null;
        try {
            connection = DriverManager.getConnection(jdbcURL,"sa","");
            statement = connection.createStatement();
            String query1 = "SELECT * FROM gtfs_stops LIMIT 5";
            resultSet = statement.executeQuery(query1);

            List<Stop> stops = new ArrayList<>();
            while (resultSet.next()) {
             stops.add(new Stop(resultSet.getLong("stop_id"),
                resultSet.getString("stop_name"),
                resultSet.getFloat("stop_lat"),
                resultSet.getFloat("stop_lon"),
                resultSet.getString("stop_code")));
         }
            System.out.println(stops);
        } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();
        } finally {
            if (resultSet != null) {
                try {
                    resultSet.close();
                } catch (SQLException e) { /* ignored */}
            }
            if (connection != null) {
                try {
                    connection.close();
                } catch (SQLException e) { /* ignored */}
            }
        }

[{id:'100027,name:'Beau Vézé',lat:'43.10094'lon:6.059851, code='CABEAN}, {id:'100028,name:'Beau Vézé',lat:'43.100803'lon:6.059965, code='CABEAS}, {id:'100029,name:'Californie',lat:'43.085365'lon:6.098909, code='CACALN}, {id:'100030,name:'Californie',lat:'43.08519'lon:6.1001363, code='CACALS}, {id:'100031,name:'Canebas',lat:'43.096413'lon:6.0689964, code='CACANE}]


A partir de Java 7 une syntaxe particulière (try-with-ressources) est possible pour fermer automatiquement certaines ressources dont les connections et les ResultSet JDBC.

In [15]:
 String jdbcURL="jdbc:h2:tcp://localhost/~/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE";

        String query1 = "SELECT * FROM gtfs_stops LIMIT 5";
        List<Stop> stops = new ArrayList<>();

        try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","")) {
            ResultSet resultSet = null;
            Statement statement = null;
            statement = connection.createStatement();
            resultSet = statement.executeQuery(query1);

            while (resultSet.next()) {
             stops.add(new Stop(resultSet.getLong("stop_id"),
                resultSet.getString("stop_name"),
                resultSet.getFloat("stop_lat"),
                resultSet.getFloat("stop_lon"),
                resultSet.getString("stop_code")));
            }
            System.out.println(stops);
        } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();
        }

[{id:'100027,name:'Beau Vézé',lat:'43.10094'lon:6.059851, code='CABEAN}, {id:'100028,name:'Beau Vézé',lat:'43.100803'lon:6.059965, code='CABEAS}, {id:'100029,name:'Californie',lat:'43.085365'lon:6.098909, code='CACALN}, {id:'100030,name:'Californie',lat:'43.08519'lon:6.1001363, code='CACALS}, {id:'100031,name:'Canebas',lat:'43.096413'lon:6.0689964, code='CACANE}]


## Types  Java/JDBC et SQL
* Malgré SQL les SGBD présentent des différences de types
* JDBC masque ces différences en définissant ses propres types SQL (constantes de la classe Types)
* Le driver assure la conversion
  * SQL vers Java lors de la lecture
  * Java vers SQL lors du passage de paramètres
* Utilisés explicitement avec les methodes getXXX() (et setXXX())
* Parfois plusieurs choix (presque tous les types SQL peuvent être retrouvés par getString())
  * CHAR et VARCHAR : getString()
  * LONGVARCHAR : getAsciiStream() et getCharacterStream()
  * BINARY et VARBINARY : getBytes()
  * LONGVARBINARY : getBinaryStream()
  * REAL : getFloat(), DOUBLE et FLOAT : getDouble()
  * DECIMAL et NUMERIC : getBigDecimal()
  * DATE : getDate(), TIME : getTime(), TIMESTAMP :getTimestamp()

* Par défaut une connexion est ouverte en « auto-
commit » :
  * un commit est automatiquement lancé après chaque requete SQL qui fait une mise à jour
* Pour un contrôle plus fin on utilise
```java
conn.setAutoCommit(false) //pour le désactiver
conn.commit() //pour valider la transaction
conn.rollback() //pour annuler la transaction
```

## Précompilation des requêtes
* Si les requêtes fabriquées à partir de String changent (paramètres) :
  * Elles sont compilées à chaque appel d'où une perte de
performances
* JDBC permet de ne compiler la requête qu'une fois (si le
SGBD le supporte)
  * En indiquant les paramètres de façon générique
  * En fixant leur valeur (sans changer la requête) au moment
de l'exécution
* Deux Statement particuliers :
  * Les requêtes paramétrées (PreparedStatement)
Les procédures stockées (CallableStatement)

In [16]:
import java.sql.PreparedStatement;
List<Stop> stops = new ArrayList<>();
try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","")) {
 PreparedStatement preparedStatement = connection.prepareStatement("SELECT * FROM gtfs_stops WHERE stop_id = ?");
 int[] ids={100034,100040,100063};
 //Juste pour illuster le cas ou une requête revient fréquement, mieux vaudrait une seule requêtes SQL
 for(int id:ids) {
    preparedStatement.setInt(1,id);
    ResultSet resultSet = preparedStatement.executeQuery();
            while (resultSet.next()) {
             stops.add(new Stop(resultSet.getLong("stop_id"),
                resultSet.getString("stop_name"),
                resultSet.getFloat("stop_lat"),
                resultSet.getFloat("stop_lon"),
                resultSet.getString("stop_code")));
            }
 }
 } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();
        }
System.out.println(stops);

[{id:'100034,name:'Carqueiranne',lat:'43.092163'lon:6.077459, code='CACARS}, {id:'100040,name:'Colle Noire (Carqueiranne)',lat:'43.1039'lon:6.0546174, code='CACOLS}, {id:'100063,name:'Canniers',lat:'43.16897'lon:6.0915866, code='CRCANS}]


## Les transactions

In [17]:
Connection connection = DriverManager.getConnection(jdbcURL, "sa","");


//Create a account table
String createAccountTableSql="CREATE TABLE account ("+
    "id SERIAL PRIMARY KEY,"+
    "name VARCHAR(100) NOT NULL,"+
    "balance DEC(15,2) NOT NULL)";
Statement statement = connection.createStatement();
statement.executeUpdate(createAccountTableSql);
statement.close();

    
//Create accounts for Alice and Bob with a prepared statement
String createAccountSql = "INSERT INTO account(name,balance) VALUES(?,?);";
PreparedStatement pstmt = connection.prepareStatement(createAccountSql);
pstmt.setString(1,"Bob");
pstmt.setInt(2, 1000);
pstmt.executeUpdate();

pstmt.setString(1,"Alice");
pstmt.setInt(2, 1000);
pstmt.executeUpdate();

pstmt.close();

PreparedStatement pstmtIncreaseAccount = connection.prepareStatement("UPDATE account SET balance = balance + ? WHERE id = ?");

//save autoCommit state
boolean autoCommit = connection.getAutoCommit();
try {
    connection.setAutoCommit(false);
    //remove 500€ to Bob
    pstmtIncreaseAccount.setInt(1,-500);
    pstmtIncreaseAccount.setInt(2,1);
    pstmtIncreaseAccount.executeUpdate();
    
    //add 500€ to Alice
    pstmtIncreaseAccount.setInt(1,500);
    pstmtIncreaseAccount.setInt(2,2);
    pstmtIncreaseAccount.executeUpdate();
    connection.commit();
} catch (SQLException exc) {
    //Cancel the whole transaction if there is a problem.
    connection.rollback();
} finally {
    //restore autoCommit state
    connection.setAutoCommit(autoCommit);
}

//Drop the table
connection.createStatement().executeUpdate("DROP TABLE account");

connection.close();

## Procédure stockées
CallableStatement permet d'appeller une procédure stockée directement sur le SGBD. 
TODO

## Metadata
* JDBC permet de récupérer des informations sur le type de données que l'on vient de récupérer par un SELECT (interface ResultSetMetaData),
* mais aussi sur la base de données elle-même (interface DatabaseMetaData)
* Les données que l'on peut récupérer avec DatabaseMetaData dépendent du SGBD avec lequel on travaille

In [18]:
//Lecture des metadonnées du serveur de base de données
import java.sql.*;

String jdbcURL="jdbc:h2:tcp://localhost/~/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE";

try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","")) {

                DatabaseMetaData metadata = connection.getMetaData();

                //Print info about the database system
                System.out.println("Database: " + metadata.getDatabaseProductName()
                        + " " + metadata.getDatabaseMajorVersion() + "." + metadata.getDatabaseMinorVersion());

                System.out.println(connection.getCatalog() + " " + connection.getSchema());

                //Retrieving the list of database names
                ResultSet tables = metadata.getTables(connection.getCatalog(),
                        connection.getSchema(),
                        "SIMPLEJDBC_PERSON", null);

                if (tables.next()) {
                    System.out.println("Table " + tables.getString("TABLE_NAME") + " already exist.");
                } else {
                    connection.createStatement().execute("CREATE TABLE \"SIMPLEJDBC_PERSON\"(" +
                            "    id INT PRIMARY KEY NOT NULL, " +
                            "    firstname VARCHAR(100))");
                    System.out.println("Table SIMPLEJDBC_PERSON created.");
                }
            } catch (SQLException e) {
                e.printStackTrace();
            }


Database: H2 1.4
gtfs_rmtt public
Table SIMPLEJDBC_PERSON created.


In [19]:
//Traitement des metadonnées d'un ResulSet
//pour la découverte du nombre, du type et du nom des colonnes. 
String jdbcURL="jdbc:h2:tcp://localhost/~/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE";

try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","");     
     ResultSet rs = connection.createStatement().executeQuery("SELECT * FROM gtfs_stops")) {

  ResultSetMetaData rsmd = rs.getMetaData();
  int nbColonnes = rsmd.getColumnCount();
  for (int i = 1; i <= nbColonnes; i++) {
    String typeColonne = rsmd.getColumnTypeName(i);
    String nomColonne = rsmd.getColumnName(i);
    System.out.println("Colonne " + i + " de nom " + nomColonne + " de type " + typeColonne);
  }
}

Colonne 1 de nom stop_id de type INTEGER
Colonne 2 de nom stop_code de type CLOB
Colonne 3 de nom stop_name de type CLOB
Colonne 4 de nom stop_lat de type DOUBLE
Colonne 5 de nom stop_lon de type DOUBLE
Colonne 6 de nom location_type de type INTEGER


## ResultSet modifiables
Il est aussi possible de paramétrer un ResultSet pour qu'il soit modifiable pendant et/ou que l'on puisse revenir en arrière.

In [20]:
try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","");     
     ResultSet rs = connection.createStatement(ResultSet.TYPE_SCROLL_INSENSITIVE,ResultSet.CONCUR_UPDATABLE)
         .executeQuery("SELECT * FROM gtfs_stops")) {
  
//to be complete    

}

## Datasources
Il est important de gérer l'ensemble des connexions qui sont ouvertes et idéalement de les réutiliser plutôt que de fermer/ouvrir. Cela peut être fait "à la main" en utilisant le concept de DataSource ou utilisant une librairie comme [Apache DBCP](http://commons.apache.org/proper/commons-dbcp/).

L'utilisation est simple on définit une classe qui paramètre une datasource qui gère un pool de connexion et fournit des connections : 

In [21]:
%%loadFromPOM
<dependency>
    <groupId>org.apache.commons</groupId>
    <artifactId>commons-dbcp2</artifactId>
    <version>2.7.0</version>
</dependency>

In [22]:
import org.apache.commons.dbcp2.BasicDataSource;

import java.sql.Connection;
import java.sql.SQLException;

public class DBCPDataSource {

    private static BasicDataSource ds = new BasicDataSource();

    static {
        ds.setUrl(jdbcURL);
        ds.setUsername("sa");
        ds.setPassword("");

        ds.setMinIdle(5);
        ds.setMaxIdle(10);
        ds.setMaxOpenPreparedStatements(100);
    }

    private DBCPDataSource() {
    }

    public static Connection getConnection() throws SQLException {
        return ds.getConnection();
    }
}

Il suffit ensuite de demander et de rendre une connexion (le close ne ferme pas forcément la connexion).

In [23]:
try (Connection connection=DBCPDataSource.getConnection()) {
    Statement statement = connection.createStatement();
    ResultSet resultSet = statement.executeQuery("SELECT * FROM gtfs_stops LIMIT 2");
    List<Stop> stops = new ArrayList<>();
    
            while (resultSet.next()) {
             stops.add(new Stop(resultSet.getLong("stop_id"),
                resultSet.getString("stop_name"),
                resultSet.getFloat("stop_lat"),
                resultSet.getFloat("stop_lon"),
                resultSet.getString("stop_code")));
            }
            System.out.println(stops);
        } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();
        }

[{id:'100027,name:'Beau Vézé',lat:'43.10094'lon:6.059851, code='CABEAN}, {id:'100028,name:'Beau Vézé',lat:'43.100803'lon:6.059965, code='CABEAS}]


## Batch Update
Pour des ajouts en masse pensez à utiliser les batch updates mais attention aux transactions.

In [24]:
try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","")) {  
     
  //Création d'une table PERSONNE
  String createAccountTableSql="CREATE TABLE PERSONNE ("+
    "id SERIAL PRIMARY KEY,"+
    "email VARCHAR(100) NOT NULL,"+
    "nom VARCHAR(100) NOT NULL,"+
    "prenom VARCHAR(100) NOT NULL)";
  Statement statement = connection.createStatement();
  statement.executeUpdate(createAccountTableSql);
  statement.close();     
     
  PreparedStatement pstmt = 
        connection.prepareStatement("INSERT INTO PERSONNE (email, nom, prenom) VALUES(?, ?, ?)");
 
  //On ajoute les exécution une à une ...
  connection.setAutoCommit(false);
 
  // On fixe les paramètres de la première requête à exécuter
  pstmt.setString( 1, "p1.n1@...");
  pstmt.setString( 2, "n1" );
  pstmt.setString( 3, "p1" );
  // Et on l'ajoute au batch
  pstmt.addBatch();
 
  // On fixe les paramètres de la seconde requête à exécuter
  pstmt.setString( 1, "p2.n2@...");
  pstmt.setString( 2, "n2" );
  pstmt.setString( 3, "p2" );
  // Et on l'ajoute au batch
  pstmt.addBatch();
 
  // On ajoute autant de requêtes que nécessaire
  //Par exemple en lisant un flux de données.


  //On créée un tableau d'entiers pour recevoir les résultats.
  //et on execute toutes les mises à jour en une fois.
  int[] affectedRecords = pstmt.executeBatch();
 
  System.out.println(Arrays.toString(affectedRecords));
    
  //On valide les changements.
  connection.commit();
    
  //Drop the table
  connection.createStatement().executeUpdate("DROP TABLE PERSONNE");    
}    

[1, 1]


# DAO
//TODO

In [25]:
//Remplacer par une Exception métier
public interface DAO<E> {
    E find(long id) throws java.sql.SQLException ;
    List<E> findAll() throws java.sql.SQLException ;
    E persist(E entity) throws java.sql.SQLException ;
    E update(E entity) throws java.sql.SQLException ;
    void remove(E entity) throws java.sql.SQLException ;
    void close() throws java.sql.SQLException ;
}


//Un record est une type de class particulier introduit avec Java 16
//qui permet de déclarer simplement des classes dédiées au transfert de données 
//cf. https://docs.oracle.com/en/java/javase/16/language/records.html
public record Personne(
    long id,
    String email,
    String nom,
    String prenom) { }

public class PersonDAO implements DAO<Personne> {
    private final Connection connection;
    private final PreparedStatement pstmt_insert;
    private final PreparedStatement pstmt_find;

    public PersonDAO(Connection connection) throws java.sql.SQLException {
        this.connection = connection; 
        pstmt_insert = connection.prepareStatement("INSERT INTO PERSONNE (email, nom, prenom) VALUES(?, ?, ?)");
        pstmt_find = connection.prepareStatement("SELECT id, email, nom, prenom FROM PERSONNE WHERE id = ?");}
    
    
    public Personne find(long id) {
        return new Personne(-1,"a.b@x.y.fr","MonNom","MonPrenom");
    }
    public List<Personne> findAll() { 
    //Completer avec un prepared statement 
        return new ArrayList<Personne>(); }
    public Personne persist(Personne personne) { 
        return personne; }
    public Personne update(Personne personne) {
        //Completer avec un prepared statement 
        return personne; }
    public void remove(Personne personne) {
        //Completer avec un prepared statement 
        }
    public void close() throws java.sql.SQLException {
        connection.close(); 
    }
}    

# Un exemple "Simple"
Pour finir voilà un exemple "Simple" d'utilisation d'une Datasource et de d'une DAO avec JDBC avec le SGBD H2.

In [25]:
%%shell
mkdir -p /home/jovyan/work/src
cd /home/jovyan/work/src
rm -rf JdbcDao
git clone https://github.com/emmanuelbruno/JdbcDao.git

Cloning into 'JdbcDao'...


Pour lancer H2 via une application Java. A ne faire qu'une seule fois.

In [26]:
%%shell
cd /home/jovyan/work/src/JdbcDao
mvn -ntp package
rm -f /home/jovyan/test.mv.db
mvn --quiet package exec:java -Dexec.mainClass=fr.univtln.bruno.jdbcdao.persistence.StartH2 &

[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------------< fr.univtln.bruno:JdbcDao >----------------------
[INFO] Building JDBC DAO 1.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-enforcer-plugin:3.0.0:enforce (enforce-java) @ JdbcDao ---
[INFO] 
[INFO] --- maven-resources-plugin:3.0.2:resources (default-resources) @ JdbcDao ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 3 resources
[INFO] 
[INFO] --- maven-compiler-plugin:3.8.0:compile (default-compile) @ JdbcDao ---
[INFO] Changes detected - recompiling the module!
[INFO] Compiling 12 source files to /home/jovyan/work/src/JdbcDao/target/classes
[INFO] 
[INFO] --- maven-resources-plugin:3.0.2:testResources (default-testResources) @ JdbcDao ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 1 resource
[INFO] 
[INFO] --- maven-compiler-plugin:3.8.0:testCompile (default-testCompile) @ JdbcDao ---
[INF

In [27]:
%%shell
cd /home/jovyan/work/src/JdbcDao
mvn --quiet package exec:java -Dexec.mainClass=fr.univtln.bruno.jdbcdao.persistence.App

Oct 15, 2021 4:40:40 PM fr.univtln.bruno.jdbcdao.persistence.daos.AbstractDAO <init>
Oct 15, 2021 4:40:40 PM fr.univtln.bruno.jdbcdao.persistence.App main
INFO:  p1 persisted Personne(id=43, nom=A, prenom=B)
Oct 15, 2021 4:40:40 PM fr.univtln.bruno.jdbcdao.persistence.App main
INFO: Personne 43 : Personne(id=43, nom=A, prenom=B)
Oct 15, 2021 4:40:40 PM fr.univtln.bruno.jdbcdao.persistence.App main
INFO: Personne -1 : MISSING !
Oct 15, 2021 4:40:40 PM fr.univtln.bruno.jdbcdao.persistence.App main
INFO: Personne 1 (new): Personne(id=43, nom=AA, prenom=BB)
Oct 15, 2021 4:40:40 PM fr.univtln.bruno.jdbcdao.persistence.App main
INFO: [Personne(id=44, nom=C, prenom=D), Personne(id=45, nom=E, prenom=F), Personne(id=46, nom=G, prenom=H), Personne(id=47, nom=I, prenom=J), Personne(id=48, nom=K, prenom=L)]
Oct 15, 2021 4:40:40 PM fr.univtln.bruno.jdbcdao.persistence.daos.AbstractDAO <init>
Oct 15, 2021 4:40:40 PM fr.univtln.bruno.jdbcdao.persistence.daos.AbstractDAO <init>
Oct 15, 2021 4:40:40 PM

Cette application peut être étudiée, testée et modifiée depuis l'éditeur Visual Code intégré ().